In [40]:
from typing import Any, Union
from pandas import DataFrame
from pandas.io.parsers import TextFileReader
%matplotlib inline
# Main Python imports and params for Mathplotlib
from datetime import datetime, timedelta

import os
import matplotlib as mpl
#if os.environ.get('DISPLAY','') == '':
    #print('no display found. Using non-interactive Agg backend')
    #mpl.use('Agg')

import matplotlib.dates as mdates
import matplotlib.pyplot as plt
from matplotlib import colors
from matplotlib import cm
import seaborn as sns


import numpy as np 
from numpy.random import rand

import time
import traceback
import math

import pandas as pd
import io
import requests

import re
import math

import json
import psycopg2

In [45]:
# download https://beam-outputs.s3.amazonaws.com/output/newyork/new-york-200k-flowCap_0.025__2020-08-09_23-27-02_gpo/ITERS/it.0/0.maps.googleapi.responses.json
# download https://beam-outputs.s3.amazonaws.com/output/newyork/new-york-200k-flowCap_0.025__2020-08-09_23-27-02_gpo/ITERS/it.10/10.maps.googleapi.responses.json

json0Url = "/path/to/0.maps.googleapi.responses.json"
json10Url = "/path/to/10.maps.googleapi.responses.json"

google_routes = []
with open(json0Url, "r") as read_file:
    google_routes = google_routes + json.load(read_file)
    
with open(json10Url, "r") as read_file:
    google_routes = google_routes + json.load(read_file)
    
google_routes_count = len(google_routes)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/aaltergot/Prj/beam/local/google_routes_db/0.maps.googleapi.responses.json'

In [42]:
def gr_point_to_point(gr_point):
    return 'POINT({} {})'.format(
        gr_point['lat'],
        gr_point['lng']
    )
    
def gr_steps_to_linestring(steps):
    return 'LINESTRING(' + ','.join(map(
        lambda step: '{} {}'.format(step['end_location']['lat'], step['end_location']['lng']), 
        [{'end_location': steps[0]['start_location']}] + steps  # hm
    )) + ')'

connection = psycopg2.connect(
    host='localhost',
    port=5432,
    user='postgres',
    password='postgres',
    dbname='google_routes_db'
)
cursor = connection.cursor()

for gr in google_routes:
    for route in gr['routes']:
        cursor.execute(
           """
           INSERT INTO gr_route (
             bound_northeast, 
             bound_southwest,
             summary, 
             copyrights
           ) VALUES (
             '{bound_northeast}', '{bound_southwest}', '{summary}', '{copyrights}'
           ) RETURNING id
           """.format(
               bound_northeast = gr_point_to_point(route['bounds']['northeast']),
               bound_southwest = gr_point_to_point(route['bounds']['southwest']),
               summary = route['summary'],
               copyrights = route['copyrights']
           )
        )
        route_id = cursor.fetchone()[0]
        
        for leg in route['legs']:
            
            duration_in_traffic = leg.get('duration_in_traffic')
            duration_in_traffic_text = 'NULL'
            if duration_in_traffic is not None:
                duration_in_traffic_text = '\'' + duration_in_traffic['text'] + '\''
                duration_in_traffic = duration_in_traffic['value']
            else:
                duration_in_traffic = 'NULL'
                
            cursor.execute(
               """
               INSERT INTO gr_route_leg (
                 route_id, distance, distance_text,
                 duration, duration_text,
                 duration_in_traffic, duration_in_traffic_text,
                 start_address, end_address,
                 start_location, 
                 end_location,
                 steps
               ) VALUES (
                 {route_id}, {distance}, '{distance_text}',
                 {duration}, '{duration_text}',
                 {duration_in_traffic}, {duration_in_traffic_text},
                 '{start_address}', '{end_address}',
                 '{start_location}', '{end_location}',
                 '{steps}'
               )
               """.format(
                   route_id = route_id, 
                   distance = leg['distance']['value'], 
                   distance_text = leg['distance']['text'],
                   duration = leg['duration']['value'], 
                   duration_text = leg['duration']['text'],
                   duration_in_traffic = duration_in_traffic, 
                   duration_in_traffic_text = duration_in_traffic_text,
                   start_address = leg['start_address'].replace("'", "''"),
                   end_address = leg['end_address'].replace("'", "''"),
                   start_location = gr_point_to_point(leg['start_location']),
                   end_location = gr_point_to_point(leg['end_location']),
                   steps = gr_steps_to_linestring(leg['steps'])
               )
            )

connection.commit()